In [27]:
import numpy as np

In [29]:
#load back the data
dir = "nn_training/"
params = []
for i in range(8):
    type_string = 'weights' * (i%2 == 0) + 'biases' * (i%2 == 1)
    params.append(np.load(dir +"nn_" + type_string + "_layer_" + str(int(i/2)) + ".npy", ))

In [30]:
#layer_sizes = np.array([784, 128, 64, 28, 10])
layer_sizes = np.array([3, 2, 1])
num_neurons = np.sum(layer_sizes)
#construct the adjacency matrix
A = np.zeros(shape = (num_neurons, num_neurons))

#each of the 784 neurons is adjacent to the 128
#each of the 128 neurons is adjacent to the 64 
#each of the 64 neurons is adjacent to the 128 and the 28
#each of the 28 neurons is adjacent to the 64 and the 10
#each of the 10 neurons is adjacent to the 28

offsets = np.r_[0, np.cumsum(layer_sizes)]   # e.g. [0, 784, 912, 976, 1004, 1014]
N = offsets[-1]                               # total neurons = 1014

# initialize adjacency
A = np.zeros((N, N), dtype=int)

# for each pair of consecutive layers i → i+1
for i in range(len(layer_sizes) - 1):
    start_i, end_i   = offsets[i],   offsets[i+1]
    start_j, end_j   = offsets[i+1], offsets[i+2]
    # connect all neurons in layer i to all in layer i+1
    A[start_i:end_i, start_j:end_j] = 1
    A[start_j:end_j, start_i:end_i] = 1  # mirror to make A symmetric
